# Submission 2: Denoising Autoencoders
Trains CNN autoencoders to denoise pneumonia images. Contains 3 CNN autoencoders:
1. Autoencoder for denoising noisy images with lambda = 25
2. Autoencoder for denoising noisy images with lambda = 50
3. Autoencoder for denoising noisy images with lambda = 75

### Load Dataset

## Evaluating results

**Why is poisson distribution the ideal one to use to simulate noise for medical images? Why not gaussian or something else? Answer in terms of relevance of medical applications.**

{Answer}

**Which one performed the best? Why do you think this is the case?**

{Answer}